In [7]:
%pwd

'c:\\Users\\abhis\\OneDrive\\Desktop\\Medical_Chat_Bot\\research'

In [8]:
import os 
os.chdir("../")

In [10]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [11]:
def load_pdf(data):
    loader = DirectoryLoader(data,
                             glob="*.pdf",
                             loader_cls=PyPDFLoader)
    
    document =loader.load()

    return document

In [12]:
extracted_data = load_pdf(data="Data/")

In [14]:
#extracted_data

In [19]:
def text_split(extracted_data):
    text_splitter= RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [20]:
text_chunks = text_split(extracted_data)
print("you good at python", len(text_chunks))

you good at python 40000


In [22]:
from langchain.embeddings import HuggingFaceBgeEmbeddings

In [25]:
def download_hugging_face_embedding():
    embeddings = HuggingFaceBgeEmbeddings(model_name = "sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [26]:
embeddings = download_hugging_face_embedding()

c:\Users\abhis\AppData\Local\anaconda3\envs\medical_bot\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\abhis\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not inst

In [27]:
query_result = embeddings.embed_query("hellow world")
print("length", len(query_result))

length 384


In [28]:
query_result

[0.0006131360423751175,
 0.18390056490898132,
 0.024796418845653534,
 0.018480323255062103,
 -0.0037284507416188717,
 -0.043419137597084045,
 0.046509817242622375,
 -0.04019160568714142,
 -0.015687400475144386,
 -0.005233929958194494,
 0.04317289963364601,
 -0.06464012712240219,
 0.0066064586862921715,
 -0.0039627705700695515,
 0.09232901781797409,
 -0.00713599706068635,
 -0.06024867296218872,
 -0.020794296637177467,
 -0.10734613984823227,
 -0.06399672478437424,
 0.07187329977750778,
 0.09736251831054688,
 0.016867324709892273,
 0.0027959260623902082,
 -0.0736025869846344,
 -0.01518813893198967,
 0.015198670327663422,
 0.02603226900100708,
 0.12841810286045074,
 -0.03059123083949089,
 -0.033475447446107864,
 0.006296554580330849,
 0.1147456094622612,
 0.02211763896048069,
 0.008302428759634495,
 0.09729631245136261,
 0.06458316743373871,
 -0.01901397854089737,
 0.007124790921807289,
 -0.055301666259765625,
 0.024569479748606682,
 -0.057817209511995316,
 -0.008697744458913803,
 0.013529

In [29]:
from dotenv import load_dotenv
load_dotenv()

True

In [31]:
PINECONE_API_KEY=os.environ.get("PINECONE_API_KEY")

In [35]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os 

pc = Pinecone(api_key=PINECONE_API_KEY)
 
index_name = "text"

pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

{
    "name": "text",
    "metric": "cosine",
    "host": "text-j6xgcjg.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [36]:
import os 
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY

In [40]:
from langchain_pinecone import PineconeVectorStore

doc =PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings
)

In [41]:
# load Existing index 
from langchain_pinecone import PineconeVectorStore

doc =PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [42]:
doc

In [43]:
retriever = doc.as_retriever(search_type = "similarity", search_kwargs={"k":3})

In [44]:
retriever_docs = retriever.invoke("what is Acne?")

In [45]:
retriever_docs

[Document(id='241f5a74-e3b1-4aef-b456-a40520c5b757', metadata={'creationdate': '2006-10-16T20:19:33+02:00', 'creator': 'Adobe Acrobat 6.0', 'moddate': '2006-10-16T22:03:45+02:00', 'page': 55.0, 'page_label': '26', 'producer': 'PDFlib+PDI 6.0.3 (SunOS)', 'source': 'Data\\The-Gale-Encyclopedia.pdf', 'total_pages': 4505.0}, page_content='Researchers, Inc. Reproduced by permission.)\n26 GALE ENCYCLOPEDIA OF MEDICINE\nAcne'),
 Document(id='f2a24bf0-b59d-41e0-a2f3-14dc09f6d176', metadata={'creationdate': '2006-10-16T20:19:33+02:00', 'creator': 'Adobe Acrobat 6.0', 'moddate': '2006-10-16T22:03:45+02:00', 'page': 55.0, 'page_label': '26', 'producer': 'PDFlib+PDI 6.0.3 (SunOS)', 'source': 'Data\\The-Gale-Encyclopedia.pdf', 'total_pages': 4505.0}, page_content='Researchers, Inc. Reproduced by permission.)\n26 GALE ENCYCLOPEDIA OF MEDICINE\nAcne'),
 Document(id='d4179d9c-4dce-4b7b-ac13-38ad5d61584f', metadata={'creationdate': '2006-10-16T20:19:33+02:00', 'creator': 'Adobe Acrobat 6.0', 'moddate':

In [46]:
from dotenv import load_dotenv
load_dotenv()

True

In [52]:
GOOGLE_API_KEY=os.environ.get("GOOGLE_API_KEY")

In [48]:
from langchain_openai import OpenAI
llm = OpenAI(temperature=0.4, max_tokens=500)

In [49]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant fro question-answering tasks." 
    "Use the following pieces of retrieved context to answer "
    "The question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human","{input}"),
    ]
)

In [50]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [ ]:
response = rag_chain.invoke({"input": "what is Acne?"})
print(response["answer"])

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}